# Chile: best comuna to place a Coffee Shop
## Coursera IBM Data Science final Capstone project

### 1- Introduction/Business problem
I live in Chile and also love coffee. I've always dreamed about putting a Coffe shop somewhere, but of course the question that arise immediately is "where?". Here, I'll try to answer that question by using some of the techniques learned on this IBM Data Science courses.
Chile is subdivided on Regions and Comunas, being this Comunas as large Neighborhoods. Please, keep in mind that. 

I'll focus on the Metropolitan Region of Chile: is where I live and the one with the most variety of business and restaurants, and of course, coffee shops. Also, is the Region where the Capital belongs, Santiago de Chile, and the one with most income of all the country. But of course, the same analysis can be applied to a different Region on Chile or even a different country, since the algorithms that I'll apply will be, of couse, generic. The only thing that maybe will be different is the data treatment, but the methodology will be the same.

### 2-Data Description
To solve the business problem described above, I'll work with the following data:
- A Wikipedia page where is the info I need: Latitude and Longitude for each Comuna in Chile, divided by Region. The page is the following: https://es.wikipedia.org/wiki/Anexo:Comunas_de_Chile
- Also, on this table, there are two more indicators tha will help me on my analysis: the Population Density (Habitants/Km^2) and the IDH (Indice de Desarrollo Humano or Human Development Index). The Density will help me to know and approximate of how many Coffee shops per habitant are there, and the IDH will help me to choose the Comuna with better incomes, since one of the indicators for that number is precisely that one. Also, the comunas with better IDH have better infraestructure, more educated habitants and more tourism, which is better for a Coffe shop.
- I'll also use the foursquare data to get, by comuna, the most common venues and then, by using K-means algorithm, group the comunas and choose the one that looks best to place the Coffee shop. Then, I'll work with the most representative group and get the number of Coffee shops existant (using again Foursquare data), get the number of coffee shops per habitant and with that number, choose the best comuna to place my shop.

I'll be using maps to visualize, first, all the comunas in the Metropolitan region and then the ones that I'll use, and also to visualize the cluster generated by the K-means algorithm. To finally make my choice I think that maybe a Choropleth map will be useful, or maybe a Horizontal plot bar will be enough: Whatever suits best to show the results to the stakeholders.